In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Cargar datos
data = pd.read_csv('data_evaluacion.csv')

In [ ]:
# Verificar nombres de las columnas
print("Columnas en el DataFrame:")
print(data.columns)

Columnas en el DataFrame:
Index(['39', 'State-gov', '77516', 'Bachelors', '13', 'Never-married',
       'Adm-clerical', 'Not-in-family', 'White', 'Male', '2174', '0', '40',
       'United-States', '<=50K'],
      dtype='object')


In [ ]:
# Preprocesamiento
data_dummies = pd.get_dummies(data, drop_first=True)

In [ ]:
# Ajustar nombre de la columna objetivo
target_column = '<=50K_>50K'

In [ ]:
# Verificar si la columna objetivo existe en los datos
if target_column not in data_dummies.columns:
    raise KeyError(f"La columna '{target_column}' no se encuentra en los datos preprocesados.")

X = data_dummies.drop(target_column, axis=1)
y = data_dummies[target_column]

In [ ]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Función para evaluar el modelo
def evaluate_model(model, X_train, y_train, X_test, y_test):
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    train_accuracy = np.mean(train_pred == y_train)
    test_accuracy = np.mean(test_pred == y_test)

    print(f"Precisión en entrenamiento: {train_accuracy:.4f}")
    print(f"Precisión en prueba: {test_accuracy:.4f}")

    if train_accuracy - test_accuracy > 0.1:
        print("Posible overfitting")
    elif test_accuracy - train_accuracy > 0.1:
        print("Posible underfitting")
    else:
        print("Modelo bien ajustado")

In [ ]:
# Evaluar modelos con diferentes kernels
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for kernel in kernels:
    svm_model = SVC(kernel=kernel, random_state=42)
    svm_model.fit(X_train_scaled, y_train)

    # Evaluar el modelo
    print(f"\nSVM con kernel {kernel}:")
    print("Confusión en entrenamiento:")
    print(confusion_matrix(y_train, svm_model.predict(X_train_scaled)))
    print("Reporte de clasificación en entrenamiento:")
    print(classification_report(y_train, svm_model.predict(X_train_scaled)))

    print("Confusión en prueba:")
    print(confusion_matrix(y_test, svm_model.predict(X_test_scaled)))
    print("Reporte de clasificación en prueba:")
    print(classification_report(y_test, svm_model.predict(X_test_scaled)))


SVM con kernel linear:
Confusión en entrenamiento:
[[24393  1595]
 [ 3460  4740]]
Reporte de clasificación en entrenamiento:
              precision    recall  f1-score   support

       False       0.88      0.94      0.91     25988
        True       0.75      0.58      0.65      8200

    accuracy                           0.85     34188
   macro avg       0.81      0.76      0.78     34188
weighted avg       0.85      0.85      0.85     34188

Confusión en prueba:
[[10485   681]
 [ 1493  1994]]
Reporte de clasificación en prueba:
              precision    recall  f1-score   support

       False       0.88      0.94      0.91     11166
        True       0.75      0.57      0.65      3487

    accuracy                           0.85     14653
   macro avg       0.81      0.76      0.78     14653
weighted avg       0.84      0.85      0.84     14653


SVM con kernel poly:
Confusión en entrenamiento:
[[24886  1102]
 [ 3933  4267]]
Reporte de clasificación en entrenamiento:
        

In [ ]:
# Evaluar underfitting y overfitting
evaluate_model(svm_model, X_train_scaled, y_train, X_test_scaled, y_test)

Precisión en entrenamiento: 0.8206
Precisión en prueba: 0.8262
Modelo bien ajustado


In [1]:
  # Guardar el modelo joblib.dump(svm_model, f'mejor_modelo_svm_{kernel}.pkl')